<img src="https://i.imgur.com/FoKB5Z5.png" align="left" width="300" height="250" title="source: imgur.com" /></a>

## Program Code: J620-002-4:2020 

## Program Name: FRONT-END SOFTWARE DEVELOPMENT

## Title : P57 - Neural Networking using Keras

#### Name: 

#### IC Number:

#### Date :

#### Introduction : 



#### Conclusion :






# Multi-Class Classification Tutorial with the Keras Deep Learning Library

Adapted from: https://machinelearningmastery.com/multi-class-classification-tutorial-keras-deep-learning-library/

## Problem Description
In this tutorial, we will use the standard machine learning problem called the iris flowers dataset.

This dataset is well studied and is a good problem for practicing on neural networks because all of the 4 input variables are numeric and have the same scale in centimeters. Each instance describes the properties of an observed flower measurements and the output variable is specific iris species.

This is a multi-class classification problem, meaning that there are more than two classes to be predicted, in fact there are three flower species. This is an important type of problem on which to practice with neural networks because the three class values require specialized handling.

The iris flower dataset is a well-studied problem and a such we can expect to achieve a model accuracy in the range of 95% to 97%. This provides a good target to aim for when developing our models.

You can download the iris flowers dataset from the [UCI Machine Learning repository](http://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data) and place it in your current working directory with the filename “iris.csv“. You can also use the iris dataset from sklearn datasets.

## Import Classes and Functions
We can begin by importing all of the classes and functions we will need in this tutorial.

This includes both the functionality we require from Keras, but also data loading from pandas as well as data preparation and model evaluation from scikit-learn.

In [4]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline
from sklearn.datasets import load_iris

## Load The Dataset
The dataset can be loaded directly. Because the output variable contains strings, it is easiest to load the data using pandas. We can then split the attributes (columns) into input variables (X) and output variables (Y).

In [5]:
# load dataset
iris = load_iris()

dataframe = pd.DataFrame(data= np.c_[iris['data'], iris['target']], columns= iris['feature_names'] + ['target'])
# dataframe = pandas.read_csv("iris.csv", header=None)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]

## Encode The Output Variable
The output variable contains three different string values.

When modeling multi-class classification problems using neural networks, it is good practice to reshape the output attribute from a vector that contains values for each class value to be a matrix with a boolean for each class value and whether or not a given instance has that class value or not.

This is called [one hot encoding](https://machinelearningmastery.com/why-one-hot-encode-data-in-machine-learning/) or creating dummy variables from a categorical variable.

For example, in this problem three class values are Iris-setosa, Iris-versicolor and Iris-virginica. If we had the observations:

In [6]:
# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)

## Define The Neural Network Model
If you are new to Keras or deep learning, see this [helpful Keras tutorial](https://machinelearningmastery.com/tutorial-first-neural-network-python-keras/).

The Keras library provides wrapper classes to allow you to use neural network models developed with Keras in scikit-learn.

There is a KerasClassifier class in Keras that can be used as an Estimator in scikit-learn, the base type of model in the library. The KerasClassifier takes the name of a function as an argument. This function must return the constructed neural network model, ready for training.

Below is a function that will create a baseline neural network for the iris classification problem. It creates a simple fully connected network with one hidden layer that contains 8 neurons.

The hidden layer uses a rectifier activation function which is a good practice. Because we used a one-hot encoding for our iris dataset, the output layer must create 3 output values, one for each class. The output value with the largest value will be taken as the class predicted by the model.

The network topology of this simple one-layer neural network can be summarized as:

    4 inputs -> [8 hidden nodes] -> 3 outputs
   
Note that we use a “softmax” activation function in the output layer. This is to ensure the output values are in the range of 0 and 1 and may be used as predicted probabilities.

Finally, the network uses the efficient Adam gradient descent optimization algorithm with a logarithmic loss function, which is called “categorical_crossentropy” in Keras.

In [16]:
# define baseline model
def baseline_model():
	# create model
	model = Sequential()
	model.add(Dense(8, input_dim=4, activation='relu'))
	model.add(Dense(4, activation='relu'))
	model.add(Dense(3, activation='softmax'))
	# Compile model
	model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
	return model

We can now create our KerasClassifier for use in scikit-learn.

We can also pass arguments in the construction of the KerasClassifier class that will be passed on to the fit() function internally used to train the neural network. Here, we pass the number of epochs as 200 and batch size as 5 to use when training the model. Debugging is also turned off when training by setting verbose to 0.

In [17]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=1)


C:\Users\Xiang Ze\AppData\Local\Temp\ipykernel_11976\116716943.py:1: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=1)


## Evaluate The Model with k-Fold Cross Validation
We can now evaluate the neural network model on our training data.

The scikit-learn has excellent capability to evaluate models using a suite of techniques. The gold standard for evaluating machine learning models is k-fold cross validation.

First we can define the model evaluation procedure. Here, we set the number of folds to be 10 (an excellent default) and to shuffle the data before partitioning it. 

In [18]:
kfold = KFold(n_splits=10, shuffle=True)


Now we can evaluate our model (estimator) on our dataset (X and dummy_y) using a 10-fold cross-validation procedure (kfold).

Evaluating the model only takes approximately 10 seconds and returns an object that describes the evaluation of the 10 constructed models for each of the splits of the dataset. *Note*: This step/process may take a while.

In [19]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Epoch 1/200
27/27 [==============================] - 1s 1ms/step - loss: 1.0829 - accuracy: 0.4000
Epoch 2/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0721 - accuracy: 0.3556
Epoch 3/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0622 - accuracy: 0.3556
Epoch 4/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0489 - accuracy: 0.3556
Epoch 5/200
27/27 [==============================] - 0s 2ms/step - loss: 1.0296 - accuracy: 0.3556
Epoch 6/200
27/27 [==============================] - 0s 2ms/step - loss: 1.0001 - accuracy: 0.3778
Epoch 7/200
27/27 [==============================] - 0s 2ms/step - loss: 0.9699 - accuracy: 0.4889
Epoch 8/200
27/27 [==============================] - 0s 1ms/step - loss: 0.9417 - accuracy: 0.4815
Epoch 9/200
27/27 [==============================] - 0s 1ms/step - loss: 0.9136 - accuracy: 0.5556
Epoch 10/200
27/27 [==============================] - 0s 2ms/step - loss: 0.8855 - accuracy: 0.5556
Epoch 11/

27/27 [==============================] - 0s 1ms/step - loss: 0.1790 - accuracy: 0.9778
Epoch 84/200
27/27 [==============================] - 0s 2ms/step - loss: 0.1763 - accuracy: 0.9778
Epoch 85/200
27/27 [==============================] - 0s 1ms/step - loss: 0.1743 - accuracy: 0.9778
Epoch 86/200
27/27 [==============================] - 0s 1ms/step - loss: 0.1695 - accuracy: 0.9778
Epoch 87/200
27/27 [==============================] - 0s 1ms/step - loss: 0.1681 - accuracy: 0.9778
Epoch 88/200
27/27 [==============================] - 0s 1ms/step - loss: 0.1702 - accuracy: 0.9704
Epoch 89/200
27/27 [==============================] - 0s 1ms/step - loss: 0.1675 - accuracy: 0.9778
Epoch 90/200
27/27 [==============================] - 0s 1ms/step - loss: 0.1611 - accuracy: 0.9704
Epoch 91/200
27/27 [==============================] - 0s 1ms/step - loss: 0.1585 - accuracy: 0.9778
Epoch 92/200
27/27 [==============================] - 0s 1ms/step - loss: 0.1578 - accuracy: 0.9778
Epoch 93/200


27/27 [==============================] - 0s 2ms/step - loss: 0.0910 - accuracy: 0.9778
Epoch 165/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0888 - accuracy: 0.9778
Epoch 166/200
27/27 [==============================] - 0s 1ms/step - loss: 0.0877 - accuracy: 0.9778
Epoch 167/200
27/27 [==============================] - 0s 1ms/step - loss: 0.0841 - accuracy: 0.9778
Epoch 168/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0856 - accuracy: 0.9778
Epoch 169/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0840 - accuracy: 0.9704
Epoch 170/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0911 - accuracy: 0.9778
Epoch 171/200
27/27 [==============================] - 0s 1ms/step - loss: 0.0855 - accuracy: 0.9778
Epoch 172/200
27/27 [==============================] - 0s 1ms/step - loss: 0.0834 - accuracy: 0.9778
Epoch 173/200
27/27 [==============================] - 0s 1ms/step - loss: 0.0821 - accuracy: 0.9778
Epoc

27/27 [==============================] - 0s 1ms/step - loss: 1.0996 - accuracy: 0.3333
Epoch 45/200
27/27 [==============================] - 0s 2ms/step - loss: 1.0995 - accuracy: 0.3333
Epoch 46/200
27/27 [==============================] - 0s 2ms/step - loss: 1.0995 - accuracy: 0.3333
Epoch 47/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0993 - accuracy: 0.3333
Epoch 48/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0992 - accuracy: 0.3333
Epoch 49/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0993 - accuracy: 0.3333
Epoch 50/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0992 - accuracy: 0.3333
Epoch 51/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0990 - accuracy: 0.3333
Epoch 52/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0989 - accuracy: 0.3333
Epoch 53/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0989 - accuracy: 0.3333
Epoch 54/200


27/27 [==============================] - 0s 1ms/step - loss: 1.0981 - accuracy: 0.3481
Epoch 126/200
27/27 [==============================] - 0s 2ms/step - loss: 1.0981 - accuracy: 0.3481
Epoch 127/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0980 - accuracy: 0.3481
Epoch 128/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0982 - accuracy: 0.3481
Epoch 129/200
27/27 [==============================] - 0s 2ms/step - loss: 1.0980 - accuracy: 0.3481
Epoch 130/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0980 - accuracy: 0.3481
Epoch 131/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0982 - accuracy: 0.3481
Epoch 132/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0981 - accuracy: 0.3481
Epoch 133/200
27/27 [==============================] - 0s 1ms/step - loss: 1.0981 - accuracy: 0.3481
Epoch 134/200
27/27 [==============================] - 0s 2ms/step - loss: 1.0980 - accuracy: 0.3481
Epoc

...

27/27 [==============================] - 0s 2ms/step - loss: 0.6215 - accuracy: 0.8741
Epoch 21/200
27/27 [==============================] - 0s 2ms/step - loss: 0.6119 - accuracy: 0.9111
Epoch 22/200
27/27 [==============================] - 0s 2ms/step - loss: 0.6017 - accuracy: 0.8889
Epoch 23/200
27/27 [==============================] - 0s 3ms/step - loss: 0.5879 - accuracy: 0.9333
Epoch 24/200
27/27 [==============================] - 0s 2ms/step - loss: 0.5781 - accuracy: 0.8889
Epoch 25/200
27/27 [==============================] - 0s 2ms/step - loss: 0.5663 - accuracy: 0.9259
Epoch 26/200
27/27 [==============================] - 0s 2ms/step - loss: 0.5561 - accuracy: 0.9481
Epoch 27/200
27/27 [==============================] - 0s 2ms/step - loss: 0.5454 - accuracy: 0.9481
Epoch 28/200
27/27 [==============================] - 0s 2ms/step - loss: 0.5362 - accuracy: 0.9333
Epoch 29/200
27/27 [==============================] - 0s 2ms/step - loss: 0.5327 - accuracy: 0.9630
Epoch 30/200


27/27 [==============================] - 0s 2ms/step - loss: 0.1650 - accuracy: 0.9704
Epoch 103/200
27/27 [==============================] - 0s 2ms/step - loss: 0.1553 - accuracy: 0.9704
Epoch 104/200
27/27 [==============================] - 0s 2ms/step - loss: 0.1589 - accuracy: 0.9704
Epoch 105/200
27/27 [==============================] - 0s 2ms/step - loss: 0.1553 - accuracy: 0.9778
Epoch 106/200
27/27 [==============================] - 0s 2ms/step - loss: 0.1537 - accuracy: 0.9704
Epoch 107/200
27/27 [==============================] - 0s 2ms/step - loss: 0.1565 - accuracy: 0.9556
Epoch 108/200
27/27 [==============================] - 0s 2ms/step - loss: 0.1506 - accuracy: 0.9778
Epoch 109/200
27/27 [==============================] - 0s 2ms/step - loss: 0.1512 - accuracy: 0.9630
Epoch 110/200
27/27 [==============================] - 0s 2ms/step - loss: 0.1464 - accuracy: 0.9778
Epoch 111/200
27/27 [==============================] - 0s 2ms/step - loss: 0.1459 - accuracy: 0.9704
Epoc

27/27 [==============================] - 0s 2ms/step - loss: 0.0965 - accuracy: 0.9704
Epoch 184/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0989 - accuracy: 0.9778
Epoch 185/200
27/27 [==============================] - 0s 2ms/step - loss: 0.1033 - accuracy: 0.9778
Epoch 186/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0935 - accuracy: 0.9778
Epoch 187/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0933 - accuracy: 0.9778
Epoch 188/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0920 - accuracy: 0.9704
Epoch 189/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0897 - accuracy: 0.9852
Epoch 190/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0948 - accuracy: 0.9704
Epoch 191/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0938 - accuracy: 0.9852
Epoch 192/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0945 - accuracy: 0.9704
Epoc

27/27 [==============================] - 0s 2ms/step - loss: 0.4041 - accuracy: 0.9407
Epoch 65/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3948 - accuracy: 0.9037
Epoch 66/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3884 - accuracy: 0.9037
Epoch 67/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3828 - accuracy: 0.9037
Epoch 68/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3749 - accuracy: 0.9111
Epoch 69/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3683 - accuracy: 0.9407
Epoch 70/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3624 - accuracy: 0.9407
Epoch 71/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3555 - accuracy: 0.9111
Epoch 72/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3463 - accuracy: 0.9333
Epoch 73/200
27/27 [==============================] - 0s 2ms/step - loss: 0.3411 - accuracy: 0.9333
Epoch 74/200


27/27 [==============================] - 0s 2ms/step - loss: 0.0909 - accuracy: 0.9704
Epoch 146/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0881 - accuracy: 0.9852
Epoch 147/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0863 - accuracy: 0.9704
Epoch 148/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0873 - accuracy: 0.9704
Epoch 149/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0876 - accuracy: 0.9704
Epoch 150/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0844 - accuracy: 0.9778
Epoch 151/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0860 - accuracy: 0.9778
Epoch 152/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0842 - accuracy: 0.9852
Epoch 153/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0833 - accuracy: 0.9778
Epoch 154/200
27/27 [==============================] - 0s 2ms/step - loss: 0.0869 - accuracy: 0.9778
Epoc